# Работа №2 от Бондаренко Алексея (bondaleksey@gmail.com)

Цель данного ноутбука на тестовом наборе данных провести:
- сравнение методов кластеризации k-means, hierarhical и DBSCAN
- осуществить визуализацию разбиения на кластеры, основывась на методы сжатия признакового пространства PCA, tSNE, UMAP 


Основные пункты задания: 

*   Предварительный анализ (EDA) и предобработка данных
*   Кластеризация и визуалиазция разбиений 
*   Интерпретация результатов

сегментация клиентов (в данном случае - банка). Крайне полезная операция, которая позволяет вам лучше познакомится со своей аудиторией, понять, на какие группы она делится и чем они характеризуются.

Построение различных вариантов кластеризаций и интерпретация результатов. 